# Labeling Interface
Run both cells. Use dropdown to load more bars, scroll to navigate, click to label.

In [5]:
import sys, pandas as pd, numpy as np, plotly.graph_objects as go, ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
from datetime import datetime
sys.path.insert(0, str(Path('..').resolve()))
from src.labeler import load_labels, save_labels
from src.config import LABEL_CLASSES, FEATURES_DIR
SYMBOL, INTERVAL = 'BTCUSDT', '1h'
df = pd.read_parquet(FEATURES_DIR / f'{SYMBOL}_{INTERVAL}_features.parquet')
df['timestamp'] = pd.to_datetime(df['open_time'])
df = df.reset_index(drop=True)
labels_data = load_labels(SYMBOL, INTERVAL)
print(f'Bars: {len(df):,} | Labels: {len(labels_data.get("labels", []))}')

Bars: 51,864 | Labels: 8


In [6]:
# === FULL INTERFACE ===
state = {'start': None, 'end': None}
colors = {0: 'rgba(255,255,0,0.3)', 1: 'rgba(0,255,0,0.3)', 2: 'rgba(255,0,0,0.3)'}

# WIDGETS
bars_dd = widgets.Dropdown(options=[('1000', 1000), ('2000', 2000), ('5000', 5000), ('10000', 10000), ('ALL', len(df))], value=5000, description='BARS:')
scroll = widgets.IntSlider(value=max(0,len(df)-5000), min=0, max=len(df)-100, step=100, description='SCROLL:', layout=widgets.Layout(width='60%'))
status = widgets.HTML('<h3>1. Click START point on chart</h3>')
start_lbl = widgets.HTML('Start: -')
end_lbl = widgets.HTML('End: -')
label_type = widgets.RadioButtons(options=[('RANGING', 0), ('UP', 1), ('DOWN', 2)], layout=widgets.Layout(width='100px'))
add_btn = widgets.Button(description='ADD', button_style='success', disabled=True)
reset_btn = widgets.Button(description='RESET', button_style='warning')
save_btn = widgets.Button(description='SAVE', button_style='primary')
del_idx = widgets.IntText(value=0, description='Del#:', layout=widgets.Layout(width='120px'))
del_btn = widgets.Button(description='DELETE', button_style='danger')
out = widgets.Output()
lbl_list = widgets.HTML('')
fig = go.FigureWidget()

def show_labels():
    h = '<b>Labels (index: type | date):</b><br>'
    for i, l in enumerate(labels_data.get('labels', [])):
        h += f"{i}: {LABEL_CLASSES[l['label']]} | {str(df.loc[l['start_idx'],'timestamp'])[:16]}<br>"
    lbl_list.value = h

def draw():
    s, e = scroll.value, min(scroll.value + bars_dd.value, len(df))
    dv = df.iloc[s:e]
    fig.data, fig.layout.shapes = [], []
    fig.add_trace(go.Scatter(x=dv['timestamp'], y=dv['close'], mode='lines', line=dict(color='white', width=1)))
    for l in labels_data.get('labels', []):
        ls, le = l['start_idx'], l['end_idx']
        if ls < e and le > s:
            fig.add_vrect(x0=df.loc[max(ls,s),'timestamp'], x1=df.loc[min(le,e)-1,'timestamp'], fillcolor=colors[l['label']], line_width=0)
    fig.update_layout(template='plotly_dark', height=500, title=f'Bars {s:,}-{e:,}', margin=dict(t=40,b=40,l=50,r=20))
    fig.data[0].on_click(click)
    show_labels()

def click(trace, points, st):
    if not points.xs: return
    idx = (df['timestamp'] - pd.to_datetime(points.xs[0])).abs().idxmin()
    if state['start'] is None:
        state['start'] = idx
        start_lbl.value = f'<b>Start:</b> {df.loc[idx,"timestamp"]}'
        status.value = '<h3 style="color:orange">2. Click END point</h3>'
        fig.add_vline(x=df.loc[idx,'timestamp'], line_color='cyan', line_width=2)
    else:
        state['end'] = idx
        if state['start'] > state['end']: state['start'], state['end'] = state['end'], state['start']
        start_lbl.value = f'<b>Start:</b> {df.loc[state["start"],"timestamp"]}'
        end_lbl.value = f'<b>End:</b> {df.loc[state["end"],"timestamp"]}'
        status.value = '<h3 style="color:lime">3. Pick type & ADD</h3>'
        add_btn.disabled = False
        fig.add_vline(x=df.loc[idx,'timestamp'], line_color='cyan', line_width=2)

def reset(b=None):
    state['start'], state['end'] = None, None
    start_lbl.value, end_lbl.value = 'Start: -', 'End: -'
    status.value = '<h3>1. Click START point</h3>'
    add_btn.disabled = True
    draw()

def add(b):
    if not state['start'] or not state['end']: return
    labels_data.setdefault('labels', []).append({'start_idx': int(state['start']), 'end_idx': int(state['end']), 'label': label_type.value, 'created_at': datetime.now().isoformat()})
    with out: clear_output(); print(f'Added {LABEL_CLASSES[label_type.value]}')
    reset()

def save(b):
    save_labels(SYMBOL, INTERVAL, labels_data)
    with out: clear_output(); print(f'Saved {len(labels_data["labels"])} labels!')

def delete(b):
    i = del_idx.value
    if 0 <= i < len(labels_data.get('labels', [])):
        labels_data['labels'].pop(i)
        save_labels(SYMBOL, INTERVAL, labels_data)
        with out: clear_output(); print(f'Deleted #{i}')
        draw()

add_btn.on_click(add); reset_btn.on_click(reset); save_btn.on_click(save); del_btn.on_click(delete)
scroll.observe(lambda c: draw(), names='value'); bars_dd.observe(lambda c: draw(), names='value')

display(widgets.VBox([
    widgets.HTML('<h2>NAVIGATION</h2>'),
    widgets.HBox([bars_dd, scroll]),
    status, fig,
    widgets.HBox([start_lbl, end_lbl]),
    widgets.HBox([label_type, add_btn, reset_btn, save_btn]),
    out,
    widgets.HTML('<hr><h2>DELETE LABELS</h2>'),
    widgets.HBox([del_idx, del_btn]),
    lbl_list
]))
draw()